In [1]:
import pandas as pd
import os

# Read all files

In [2]:
all_data=pd.DataFrame()
for file in os.listdir('Data/Raw'):
    file_csv = pd.read_csv(f'Data/Raw/{file}')
    all_data = pd.concat([all_data,file_csv])
all_data.reset_index(drop=True,inplace=True)
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


# Data Cleaning

In [3]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [4]:
all_data[all_data['Order ID'].isna()].notna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

In [5]:
# drop nan
all_data=all_data[all_data['Order ID'].notna()]

In [6]:
all_data[all_data.duplicated(keep=False)].sort_values(by='Order ID')

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
68421,142071,AA Batteries (4-pack),1,3.84,01/17/19 23:02,"131 2nd St, Boston, MA 02215"
68422,142071,AA Batteries (4-pack),1,3.84,01/17/19 23:02,"131 2nd St, Boston, MA 02215"
71672,145143,Lightning Charging Cable,1,14.95,01/06/19 03:01,"182 Jefferson St, San Francisco, CA 94016"
71673,145143,Lightning Charging Cable,1,14.95,01/06/19 03:01,"182 Jefferson St, San Francisco, CA 94016"
73357,146765,Google Phone,1,600,01/21/19 11:23,"918 Highland St, New York City, NY 10001"
...,...,...,...,...,...,...
3209,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
3618,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
130303,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2893,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [7]:
# drop duplicated rows
all_data.drop_duplicates(keep='first',inplace=True)

In [8]:
all_data[~all_data['Quantity Ordered'].str.isnumeric()]

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [9]:
# drop inconsistent rows
all_data.drop(all_data[~all_data['Quantity Ordered'].str.isnumeric()].index,inplace=True)

In [10]:
# change data types
all_data['Quantity Ordered']=pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each']=pd.to_numeric(all_data['Price Each'])
all_data['Order Date']=pd.to_datetime(all_data['Order Date'])

In [11]:
# renew columns
new_columns={
    'Order ID':'ID',
    'Quantity Ordered':'Quantity_ordered',
    'Price Each':'Price_each',
    'Order Date':'Order_date',
    'Purchase Address':'Purchase_address',
}
all_data.rename(new_columns,axis=1,inplace=True)
all_data.head()

,ID,Product,Quantity_ordered,Price_each,Order_date,Purchase_address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"


In [12]:
# Parsing the address
all_data['Street']= [address[0] for address in all_data['Purchase_address'].str.split(',')]
all_data['City']= [address[1] for address in all_data['Purchase_address'].str.split(',')]
all_data['State']= [address[2] for address in all_data['Purchase_address'].str.split(',')]

all_data['Zip_code']= [address[1] for address in all_data['State'].str.split()]
all_data['State']= [address[0] for address in all_data['State'].str.split()]

In [13]:
all_data.drop('Purchase_address',axis=1,inplace=True)
all_data.head()

,ID,Product,Quantity_ordered,Price_each,Order_date,Street,City,State,Zip_code
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,917 1st St,Dallas,TX,75001
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,682 Chestnut St,Boston,MA,02215
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,669 Spruce St,Los Angeles,CA,90001
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,333 8th St,Los Angeles,CA,90001


In [14]:
all_data.to_csv('./Data/Clean/clean_data.csv',index=False)